In [14]:
import pandas as pd
import numpy as np

## read datafiles
- C-18 for language population
- C-13 for particular age-range population from a state

In [15]:
c18=pd.read_excel('datasets/C-18.xlsx',skiprows=6,header=None,engine='openpyxl')

In [16]:
c13=pd.read_excel('datasets/C-13.xls',skiprows=7,header=None)

### particular age groups are
- 5-9
- 10-14
- 15-19
- 20-24
- 25-29
- 30-49
- 50-69
- 70+
- Age not stated

## obtain useful data from C-13 and C-18 for age-groups
- first get particular state names for identifying specific states
- get particular age-groups from C-18 file
- make list of particular age group row/col for a particular states
- now just simply iterate through each state to get relevent data and store it into a csv file
    - to get total pop of particular age-range I have used C-13 file
    - to get total pop that speaks more than 3 languages from a state in a particular age-range I have used C-18 file 

In [17]:
# STATE_NAMES=[list(np.unique(c18.iloc[:,2].values))]
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)
AGE_GROUPS=list(c18.iloc[1:10,4].values)
# although it is a bit of manual work but it is worth the efforts
AGE_GROUP_RANGES=[list(range(5,10)),list(range(10,15)),list(range(15,20)),list(range(20,25)),list(range(25,30)),list(range(30,50)),list(range(50,70)),list(range(70,100))+['100+'],['Age not stated']]

In [18]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    for j,age_grp in enumerate(AGE_GROUPS):
        # this list is to get only the years in the particular age-group

        true_false_list=[]
        for single_year_age in c13.iloc[:,4].values:
            if single_year_age in AGE_GROUP_RANGES[j]:
                true_false_list.append(True)
            else:
                true_false_list.append(False)

        # here i is the state code
        male_pop=c13[(c13.loc[:,1]==i) & (true_false_list)].iloc[:,6].values.sum()
        female_pop=c13[(c13.loc[:,1]==i) & (true_false_list)].iloc[:,7].values.sum()
        
        # tri
        tri_male=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,9]
        tri_female=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,10]

        #bi
        bi_male=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,6] - tri_male
        bi_female=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,7] - tri_female

        #uni
        uni_male=male_pop-bi_male-tri_male
        uni_female=female_pop-bi_female-tri_female

        item={
            'state-code':i,
            'state-name':state,
            'age-group':age_grp,
            'age-group-male-pop':male_pop,
            'age-group-female-pop':female_pop,
            'tri-male-ratio':tri_male/male_pop,
            'tri-female-ratio':tri_female/female_pop,
            'bi-male-ratio':bi_male/male_pop,
            'bi-female-ratio':bi_female/female_pop,
            'uni-male-ratio':uni_male/male_pop,
            'uni-female-ratio':uni_female/female_pop
        }

        useful_data.append(item)

In [19]:
df=pd.DataFrame(useful_data)

## age-analysis 
- get highest ratio age-group for a state and store it into csv file
- above process can be repeated for all parts of the question

In [20]:
tri_list=[]
bi_list=[]
uni_list=[]
for i in range(36):
    male_values=df[df['state-code']==i].sort_values(by='tri-male-ratio',ascending=False).iloc[0,[2,5]].values
    female_values=df[df['state-code']==i].sort_values(by='tri-male-ratio',ascending=False).iloc[0,[2,6]].values
    tri_item={
        'state/ut':i,
        'age-group-males':male_values[0],
        'ratio-males':male_values[1],
        'age-group-females':female_values[0],
        'ratio-females':female_values[1]
    }

    tri_list.append(tri_item)

    male_values=df[df['state-code']==i].sort_values(by='bi-male-ratio',ascending=False).iloc[0,[2,7]].values
    female_values=df[df['state-code']==i].sort_values(by='bi-male-ratio',ascending=False).iloc[0,[2,8]].values
    bi_item={
        'state/ut':i,
        'age-group-males':male_values[0],
        'ratio-males':male_values[1],
        'age-group-females':female_values[0],
        'ratio-females':female_values[1]
    }

    bi_list.append(bi_item)

    male_values=df[df['state-code']==i].sort_values(by='uni-male-ratio',ascending=False).iloc[0,[2,9]].values
    female_values=df[df['state-code']==i].sort_values(by='uni-male-ratio',ascending=False).iloc[0,[2,10]].values
    uni_item={
        'state/ut':i,
        'age-group-males':male_values[0],
        'ratio-males':male_values[1],
        'age-group-females':female_values[0],
        'ratio-females':female_values[1]
    }

    uni_list.append(uni_item)

- convert into pandas dataframes and store into CSVs

In [21]:
tri_df=pd.DataFrame(tri_list)
bi_df=pd.DataFrame(bi_list)
uni_df=pd.DataFrame(uni_list)

In [22]:
tri_df.to_csv('outputs/age-gender-a.csv',index=False)
bi_df.to_csv('outputs/age-gender-b.csv',index=False)
uni_df.to_csv('outputs/age-gender-c.csv',index=False)

## observations

- in almost all states(and all cases) both highest ratio female and male age-groups are same.
- interestingly in only one language case for all states '5-9' age group dominates, and it is also quite intutive; since at that early stage in life children only speak their mother toung only 

In [23]:
uni_df

,state/ut,age-group-males,ratio-males,age-group-females,ratio-females
0,0,5-9,0.876821,5-9,0.876585
1,1,5-9,0.685737,5-9,0.696543
2,2,5-9,0.887130,5-9,0.889550
3,3,5-9,0.801164,5-9,0.802787
4,4,5-9,0.713938,5-9,0.725435
5,5,5-9,0.903684,5-9,0.903566
6,6,5-9,0.906105,5-9,0.907915
7,7,5-9,0.844588,5-9,0.844506
8,8,5-9,0.939905,5-9,0.941008
9,9,5-9,0.954013,5-9,0.953154
